# Lexique des émotions

#### Chargement des bibliothèques

In [1]:
import os
import json
import spacy
import csv
import numpy as np
import re
from collections import Counter
from collections import defaultdict
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd


nlp = spacy.load('fr_core_news_lg')


#### Chargement du fichier source

In [2]:
csv_original = "./lexique/FEEL.csv"

# Charger les données à partir du fichier CSV dans un DataFrame
df = pd.read_csv(csv_original, delimiter=";", index_col=0)

df.head(1590)

,word,polarity,joy,fear,sadness,anger,surprise,disgust
id,,,,,,,,
1,à ce endroit là,positive,0,0,0,0,0,0
2,à le hâte,negative,0,1,0,0,1,0
3,à part,negative,0,0,1,0,0,0
4,à pic,negative,0,1,0,0,0,0
5,à rallonge,negative,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
1586,bon gorgée,negative,0,0,0,0,0,1
1587,école maternel,positive,0,0,0,0,0,0
1588,un foi|fois tout le deux moi|mois,positive,0,0,0,0,0,0


In [3]:
len(df)

14127

In [4]:
# Sélectionner les colonnes d'intérêt pour les émotions
emotions_df = df.iloc[:, 1:] 

# Créer une matrice 8x8 remplie de zéros
matrix = pd.DataFrame(0, index=emotions_df.columns, columns=emotions_df.columns)

# Parcourir les termes et mettre à jour la matrice avec les occurrences des paires émotion-polarité
for term, row in emotions_df.iterrows():
    for emotion1 in emotions_df.columns:
        if row[emotion1] != 0:
            for emotion2 in emotions_df.columns:
                if row[emotion2] != 0:
                    matrix.loc[emotion1, emotion2] += 1

# Affichage de la matrice
display(matrix)


,polarity,joy,fear,sadness,anger,surprise,disgust
polarity,14127,521,3199,2512,2103,1182,2013
joy,521,521,0,0,0,0,0
fear,3199,0,3199,1532,1335,657,909
sadness,2512,0,1532,2512,932,291,836
anger,2103,0,1335,932,2103,355,889
surprise,1182,0,657,291,355,1182,133
disgust,2013,0,909,836,889,133,2013


#### Split du champ ``polarity``

In [16]:
# Traitement de la polarité
polarity_path = "./lexique/emotion_dictionary.csv"

with open(csv_original, 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file, delimiter=';')
    
    with open(polarity_path, 'w', encoding='utf-8', newline='') as out_file:
        fieldnames = ['word', 'positive', 'negative', 'joy', 'fear', 'sadness', 'anger', 'surprise', 'disgust']
        csv_writer = csv.DictWriter(out_file, fieldnames=fieldnames, delimiter=';')
        csv_writer.writeheader()
        
        for row in csv_reader:
            word = row['word']
            polarity = row['polarity']
            emotions = [int(row['joy']), int(row['fear']), int(row['sadness']),
                        int(row['anger']), int(row['surprise']), int(row['disgust'])]
            
            positive = 1 if polarity == 'positive' else 0
            negative = 1 if polarity == 'negative' else 0
            
            csv_writer.writerow({
                'word': word,
                'positive': positive,
                'negative': negative,
                'joy': emotions[0],
                'fear': emotions[1],
                'sadness': emotions[2],
                'anger': emotions[3],
                'surprise': emotions[4],
                'disgust': emotions[5]
            })


In [17]:
csv_clean = "./lexique/emotion_dictionary.csv"

# Charger les données à partir du fichier CSV dans un DataFrame
df = pd.read_csv(csv_clean, delimiter=";", index_col=0)

# Sélectionner les colonnes d'intérêt pour les émotions en utilisant l'index numérique
emotions_df = df.iloc[:, 0:]  # Ici, nous supposons que les colonnes d'intérêt commencent à l'index 1 (positivity) jusqu'à la fin

# Créer une matrice 8x8 remplie de zéros
matrix = pd.DataFrame(0, index=emotions_df.columns, columns=emotions_df.columns)

# Parcourir les termes et mettre à jour la matrice avec les occurrences des paires émotion-polarité
for term, row in emotions_df.iterrows():
    for emotion1 in emotions_df.columns:
        if row[emotion1] != 0:
            for emotion2 in emotions_df.columns:
                if row[emotion2] != 0:
                    matrix.loc[emotion1, emotion2] += 1

# Affichage de la matrice
display(matrix)


,positive,negative,joy,fear,sadness,anger,surprise,disgust
positive,8423,0,514,223,131,120,435,91
negative,0,5704,7,2976,2381,1983,747,1922
joy,514,7,521,0,0,0,0,0
fear,223,2976,0,3199,1532,1335,657,909
sadness,131,2381,0,1532,2512,932,291,836
anger,120,1983,0,1335,932,2103,355,889
surprise,435,747,0,657,291,355,1182,133
disgust,91,1922,0,909,836,889,133,2013


#### Nettoyage "automatique" du lexique 

In [18]:
polarity_path = "./lexique/emotion_dictionary.csv"
clean_path = "./lexique/emotion_dictionary_clean.csv"

modified_rows = []
modified_words = []  # Liste pour stocker les mots modifiés

with open(polarity_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file, delimiter=';')
    
    for row in csv_reader:
        word = row['word']
        original_word = word
        word = word.replace("?", "oe")
        word = re.sub(r"\bl ", "l'", word)
        word = re.sub(r"\bd ", "d'", word)
        word = re.sub(r"\bs ", "s'", word)
        word = word.replace(" qch", "")
        word = word.replace(" qqn", "")
        word = word.replace("Ã¯", "ï")
        word = word.replace("Ã»", "û")
        word_parts = [part.split('|')[1] if '|' in part else part for part in word.split()]
        new_word = ' '.join(word_parts)
        row['word'] = new_word
        modified_rows.append(row)

        if original_word != new_word:
            modified_words.append({'original_word': original_word, 'modified_word': new_word})

with open(clean_path, 'w', encoding='utf-8', newline='') as file:
    fieldnames = csv_reader.fieldnames
    csv_writer = csv.DictWriter(file, delimiter=';', fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(modified_rows)

# Afficher le nombre de modifications
nb_modifications = len(modified_words)
print(f"Nombre de modifications : {nb_modifications}")
print("--------------------")

# Afficher les mots modifiés
for modified_word in modified_words:
    print(f"Mot d'origine : {modified_word['original_word']}")
    print(f"Mot modifié : {modified_word['modified_word']}")
    print("--------------------")

Nombre de modifications : 294
--------------------
Mot d'origine : coiffeur|coiffeuse
Mot modifié : coiffeuse
--------------------
Mot d'origine : décombre|décombres
Mot modifié : décombres
--------------------
Mot d'origine : dégoÃ»ter
Mot modifié : dégoûter
--------------------
Mot d'origine : directeur|directrice de le photographie
Mot modifié : directrice de le photographie
--------------------
Mot d'origine : dormeur|dormeuse
Mot modifié : dormeuse
--------------------
Mot d'origine : du moi|mois précédent
Mot modifié : du mois précédent
--------------------
Mot d'origine : effrayer|effrayer
Mot modifié : effrayer
--------------------
Mot d'origine : empaqueteur|empaqueteuse
Mot modifié : empaqueteuse
--------------------
Mot d'origine : environ|environs
Mot modifié : environs
--------------------
Mot d'origine : folle|fou
Mot modifié : fou
--------------------
Mot d'origine : folle|fou de joie
Mot modifié : fou de joie
--------------------
Mot d'origine : folle|fou furieux
Mot mo

In [20]:
# Compter le nombre de lignes dans le fichier CSV original
with open(polarity_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    num_lines_polarity = len(list(csv_reader)) - 1

# Compter le nombre de lignes dans le fichier CSV nettoyé
with open(clean_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    num_lines_clean = len(list(csv_reader)) - 1

# Afficher le nombre de lignes
print("Nombre de lignes :", num_lines_polarity)
print("Nombre de lignes dans le fichier traité :", num_lines_clean)


Nombre de lignes : 14127
Nombre de lignes dans le fichier traité : 14127


In [4]:
def load_csv_content(csv_path):
    csv_content = []
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile, delimiter=';')
        for row in csv_reader:
            word = row['word']
            polarity = np.array([int(row['positive']), int(row['negative'])])
            emotions = np.array([int(row['joy']), int(row['fear']), int(row['sadness']),
                                 int(row['anger']), int(row['surprise']), int(row['disgust'])])
            csv_content.append({'word': word, 'polarity': polarity, 'emotions': emotions})
    return csv_content


In [5]:
def load_dictionary(csv_path):
    emotion_dict = {}

    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile, delimiter=';')
        for row in csv_reader:
            word = row['word']
            polarity = np.array([int(row['positive']), int(row['negative'])])
            emotions = np.array([int(row['joy']), int(row['fear']), int(row['sadness']),
                                 int(row['anger']), int(row['surprise']), int(row['disgust'])])

            if word in emotion_dict:
                # Fusionner les vecteurs d'émotions des doublons en prenant la valeur maximale
                emotion_dict[word]['polarity'] = np.maximum(emotion_dict[word]['polarity'], polarity)
                emotion_dict[word]['emotions'] = np.maximum(emotion_dict[word]['emotions'], emotions)
            else:
                emotion_dict[word] = {'polarity': polarity, 'emotions': emotions}

    return emotion_dict


#### Correction manuelle

In [6]:
polarity_path = "./lexique/emotion_dictionary.csv"
clean_path = "./lexique/emotion_dictionary_clean.csv"
manual_path = "./lexique/emotion_dictionary_clean_manual.csv"

# Comptage des lignes du CSV
csv_polarity = load_csv_content(polarity_path)
csv_clean = load_csv_content(clean_path)
csv_manual = load_csv_content(manual_path)

num_csv_polarity = len(csv_polarity)
num_csv_clean = len(csv_clean)
num_csv_manual = len(csv_manual)

# Comptage des clés du dictionnaire
dict_polarity = load_dictionary(polarity_path) 
dict_clean = load_dictionary(clean_path)
dict_manual = load_dictionary(manual_path)

num_dict_polarity = len(dict_polarity)
num_dict_clean = len(dict_clean)
num_dict_manual = len(dict_manual)

print("Nombre de lignes (sans les en-têtes) :")
print("---------------------")
print(f"CSV - Gestion des polarité : {num_csv_polarity}")
print(f"CSV - Nettoyage automatique : {num_csv_clean}")
print(f"CSV - Nettoyage manuel : {num_csv_manual}")
print("---------------------")
print(f"Dictionnaire - Gestion des polarités : {num_dict_polarity}")
print(f"Dictionnaire - Nettoyage automatique : {num_dict_clean}")
print(f"Dictionnaire - Nettoyage manuel : {num_dict_manual}")

Nombre de lignes (sans les en-têtes) :
---------------------
CSV - Gestion des polarité : 14127
CSV - Nettoyage automatique : 14127
CSV - Nettoyage manuel : 14127
---------------------
Dictionnaire - Gestion des polarités : 14126
Dictionnaire - Nettoyage automatique : 14110
Dictionnaire - Nettoyage manuel : 14072


In [7]:
def find_duplicates(csv_file):
    duplicates = {}
    for entry in csv_file:
        word = entry['word']
        if word in duplicates:
            duplicates[word]['polarity'].append(entry['polarity'])
            duplicates[word]['emotions'].append(entry['emotions'])
        else:
            duplicates[word] = {'polarity': [entry['polarity']], 'emotions': [entry['emotions']]}
    return duplicates


In [8]:
# Charger le fichier CSV
manual_path = "./lexique/emotion_dictionary_clean_manual.csv"

csv_manual = load_csv_content(manual_path)

# Trouver les doublons dans le dictionnaire
duplicates = find_duplicates(csv_manual)

for word, values in duplicates.items():
    if len(values['polarity']) > 1:
        print("---------------------")
        print("'"+ word + "'")
        print("---------------------")
        for i in range(len(values['polarity'])):
            print("Polarité :", values['polarity'][i])
            print("Vecteur d'émotions :", values['emotions'][i])


---------------------
'acclamer'
---------------------
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 1 0]
---------------------
'accroître'
---------------------
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
---------------------
'bien'
---------------------
Polarité : [1 0]
Vecteur d'émotions : [1 0 0 0 0 0]
Polarité : [1 0]
Vecteur d'émotions : [0 1 0 0 1 0]
---------------------
'défloration'
---------------------
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
---------------------
'dégoût'
---------------------
Polarité : [0 1]
Vecteur d'émotions : [0 1 1 1 0 1]
Polarité : [0 1]
Vecteur d'émotions : [0 1 0 1 0 1]
---------------------
'dégoûtant'
---------------------
Polarité : [0 1]
Vecteur d'émotions : [0 1 0 1 0 1]
Polarité : [0 1]
Vecteur d'émotions : [0 1 0 1 0 1]
---------------------
'dégoûter'
--------------

#### Vérification des doublons après lemmatisation du dictionnaire

In [12]:
def tokenize_and_lemmatize(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct]
    return tokens


In [13]:
manual_path = "./lexique/emotion_dictionary_clean_manual.csv"
lemma_path = "./lexique/emotion_dictionary_lemmatized.csv"

def write_lemmatized_dictionary(emotion_dict_path, output_file):
    with open(emotion_dict_path, newline='', encoding='utf-8') as csvfile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(csvfile, delimiter=';')
        fieldnames = csv_reader.fieldnames
        csv_writer = csv.DictWriter(outfile, delimiter=';', fieldnames=fieldnames)
        csv_writer.writeheader()

        for row in csv_reader:
            word = row['word']
            lemmatized_words = tokenize_and_lemmatize(word.lower())
            lemmatized_expression = ' '.join(lemmatized_words)
            row['word'] = lemmatized_expression
            csv_writer.writerow(row)

# Écriture du dictionnaire d'émotions lemmatisé dans un fichier de sortie
write_lemmatized_dictionary(manual_path, lemma_path)


In [16]:
polarity_path = "./lexique/emotion_dictionary.csv"
clean_path = "./lexique/emotion_dictionary_clean.csv"
manual_path = "./lexique/emotion_dictionary_clean_manual.csv"
lemma_path = "./lexique/emotion_dictionary_lemmatized.csv"

# Comptage des lignes du CSV
csv_polarity = load_csv_content(polarity_path)
csv_clean = load_csv_content(clean_path)
csv_manual = load_csv_content(manual_path)
csv_lemma = load_csv_content(lemma_path)

num_csv_polarity = len(csv_polarity)
num_csv_clean = len(csv_clean)
num_csv_manual = len(csv_manual)
num_csv_lemma = len(csv_lemma)

# Comptage des clés du dictionnaire
dict_polarity = load_dictionary(polarity_path) 
dict_clean = load_dictionary(clean_path)
dict_manual = load_dictionary(manual_path)
dict_lemma = load_dictionary(lemma_path)

num_dict_polarity = len(dict_polarity)
num_dict_clean = len(dict_clean)
num_dict_manual = len(dict_manual)
num_dict_lemma = len(dict_lemma)

print("Nombre de lignes (sans les en-têtes) :")
print("---------------------")
print(f"CSV - Gestion des polarités : {num_csv_polarity}")
print(f"CSV - Nettoyage automatique : {num_csv_clean}")
print(f"CSV - Nettoyage manuel : {num_csv_manual}")
print(f"CSV - Lemmatisation spaCy : {num_csv_lemma}")
print("---------------------")
print(f"Dictionnaire - Gestion des polarités : {num_dict_polarity}")
print(f"Dictionnaire - Nettoyage automatique : {num_dict_clean}")
print(f"Dictionnaire - Nettoyage manuel : {num_dict_manual}")
print(f"Dictionnaire - Lemmatisation spaCy : {num_dict_lemma}")

Nombre de lignes (sans les en-têtes) :
---------------------
CSV - Gestion des polarités : 14127
CSV - Nettoyage automatique : 14127
CSV - Nettoyage manuel : 14127
CSV - Lemmatisation spaCy : 14125
---------------------
Dictionnaire - Gestion des polarités : 14126
Dictionnaire - Nettoyage automatique : 14110
Dictionnaire - Nettoyage manuel : 14072
Dictionnaire - Lemmatisation spaCy : 13658


Attention, à ce stade, les deux mots "avoir" (venant de "aura" et "as") ont supprimés manuellement.

In [17]:
# Trouver les doublons dans le dictionnaire
duplicates = find_duplicates(csv_lemma)

# Imprimer les doublons et leurs vecteurs correspondants
for word, values in duplicates.items():
    if len(values['polarity']) > 1:
        print("---------------------")
        print("'" + word + "'")
        print("---------------------")
        for i in range(len(values['polarity'])):
            print("Polarité :", values['polarity'][i])
            print("Vecteur d'émotions :", values['emotions'][i])
            

---------------------
'abandonner'
---------------------
Polarité : [0 1]
Vecteur d'émotions : [0 1 1 0 0 0]
Polarité : [0 1]
Vecteur d'émotions : [0 1 1 1 0 1]
---------------------
'abattre'
---------------------
Polarité : [0 1]
Vecteur d'émotions : [0 0 0 0 0 1]
Polarité : [0 1]
Vecteur d'émotions : [0 1 1 1 1 0]
---------------------
'abîmer'
---------------------
Polarité : [0 1]
Vecteur d'émotions : [0 1 1 0 0 0]
Polarité : [0 1]
Vecteur d'émotions : [0 1 1 1 0 0]
---------------------
'absorber'
---------------------
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 1 0]
---------------------
'acclamer'
---------------------
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 1 0]
---------------------
'accepter'
---------------------
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
Polarité : [1 0]
Vecteur d'émotions : [0 0 0 0 0 0]
---------------------
'accroître'
-------------

In [18]:
dict_lemma = load_dictionary(lemma_path)

pprint(dict_lemma)


{'@card@ kilo': {'emotions': array([0, 0, 0, 0, 0, 0]),
                 'polarity': array([1, 0])},
 'abaissement': {'emotions': array([0, 0, 1, 0, 0, 0]),
                 'polarity': array([0, 1])},
 'abaisser': {'emotions': array([0, 0, 1, 0, 0, 0]), 'polarity': array([0, 1])},
 'abandon': {'emotions': array([0, 1, 1, 1, 1, 0]), 'polarity': array([0, 1])},
 'abandonner': {'emotions': array([0, 1, 1, 1, 0, 1]),
                'polarity': array([0, 1])},
 'abasourdir': {'emotions': array([0, 0, 0, 0, 1, 0]),
                'polarity': array([0, 1])},
 'abattoir': {'emotions': array([0, 1, 1, 1, 0, 1]), 'polarity': array([0, 1])},
 'abattre': {'emotions': array([0, 1, 1, 1, 1, 1]), 'polarity': array([0, 1])},
 'abbé': {'emotions': array([0, 1, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'abcès': {'emotions': array([0, 1, 1, 0, 0, 1]), 'polarity': array([0, 1])},
 'abdiquer': {'emotions': array([0, 1, 1, 0, 0, 0]), 'polarity': array([0, 1])},
 'abdomen': {'emotions': array([0, 0, 0, 0

 'baliverne': {'emotions': array([0, 0, 1, 1, 0, 1]),
               'polarity': array([0, 1])},
 'ballade': {'emotions': array([1, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'ballast': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'ballaster': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'ballet': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'ballon': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'ballon de basket': {'emotions': array([1, 0, 0, 0, 0, 0]),
                      'polarity': array([1, 0])},
 'ballot': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'balsamine': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'balsamique': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'balustrade': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'b

 'compléter': {'emotions': array([0, 0, 0, 0, 1, 0]),
               'polarity': array([1, 0])},
 'comportement': {'emotions': array([0, 0, 0, 0, 0, 0]),
                  'polarity': array([1, 0])},
 'comporter': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'composant': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'composante': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'composer': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'composer de constituer de': {'emotions': array([0, 0, 0, 0, 0, 0]),
                               'polarity': array([1, 0])},
 'composer un numéro': {'emotions': array([0, 0, 0, 0, 0, 0]),
                        'polarity': array([1, 0])},
 'composite': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'compositeur': {'emotions': array([0, 0, 0, 0, 0, 0]),
                 'polarity': 

 'dédommager': {'emotions': array([0, 0, 0, 0, 1, 0]),
                'polarity': array([1, 0])},
 'déductif': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'déduction': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'déduire': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'déesse': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'défaillir': {'emotions': array([0, 1, 1, 1, 0, 0]),
               'polarity': array([0, 1])},
 'défaire': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'défaire son valise': {'emotions': array([0, 0, 0, 0, 0, 0]),
                        'polarity': array([1, 0])},
 'défaite': {'emotions': array([0, 0, 0, 1, 0, 0]), 'polarity': array([0, 1])},
 'défaut de paiement': {'emotions': array([0, 1, 1, 0, 0, 0]),
                        'polarity': array([0, 1])},
 'défavorable': {'emotions': array([0, 1, 1, 1, 0, 1]),
                 'pol

 'fourmiller': {'emotions': array([0, 1, 0, 1, 0, 0]),
                'polarity': array([0, 1])},
 'fournaise': {'emotions': array([0, 0, 0, 1, 0, 0]),
               'polarity': array([0, 1])},
 'fourner': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'fournir': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'fournisseur': {'emotions': array([0, 0, 0, 0, 0, 0]),
                 'polarity': array([1, 0])},
 'fourniture': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'fourniture de bureau': {'emotions': array([0, 0, 0, 0, 0, 0]),
                          'polarity': array([1, 0])},
 'fourr tout': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([0, 1])},
 'fourreau': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'fourrer': {'emotions': array([0, 1, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'fourrure': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': 

...

 'méditation': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'méditer': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'méditerranéen': {'emotions': array([0, 0, 0, 0, 0, 0]),
                   'polarity': array([1, 0])},
 'médium': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'médiéval': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'méduser': {'emotions': array([0, 1, 0, 0, 1, 1]), 'polarity': array([0, 1])},
 'méfait': {'emotions': array([0, 1, 1, 1, 0, 1]), 'polarity': array([0, 1])},
 'méfiance': {'emotions': array([0, 1, 0, 1, 0, 1]), 'polarity': array([0, 1])},
 'méfiant': {'emotions': array([0, 1, 0, 1, 1, 0]), 'polarity': array([0, 1])},
 'mélancolie': {'emotions': array([0, 0, 1, 0, 0, 0]),
                'polarity': array([0, 1])},
 'mélancolique': {'emotions': array([0, 0, 1, 0, 0, 0]),
                  'polarity': array([0, 1])},
 'mélancoliquement': {'emotions': 

 'poste vacant': {'emotions': array([1, 0, 0, 0, 0, 0]),
                  'polarity': array([1, 0])},
 'poster': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'posthume': {'emotions': array([0, 0, 1, 0, 0, 0]), 'polarity': array([0, 1])},
 'postillonner': {'emotions': array([0, 0, 0, 0, 0, 1]),
                  'polarity': array([0, 1])},
 'postulat': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'postuler': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'postuler à': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'posture': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'postérieur': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([0, 1])},
 'postérieurement': {'emotions': array([0, 0, 0, 0, 0, 0]),
                     'polarity': array([0, 1])},
 'postériorité': {'emotions': array([0, 0, 0, 0, 0, 0]),
                  'polarity'

 'rond point': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'rondin': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'ronflement': {'emotions': array([0, 0, 0, 1, 0, 1]),
                'polarity': array([0, 1])},
 'ronfler': {'emotions': array([0, 0, 0, 1, 0, 1]), 'polarity': array([0, 1])},
 'ronronnement': {'emotions': array([0, 0, 0, 0, 0, 0]),
                  'polarity': array([1, 0])},
 'ronronner': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'roquette': {'emotions': array([0, 1, 0, 1, 0, 0]), 'polarity': array([0, 1])},
 'rosaire': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'rose': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'roseau': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'roser': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'rosette': {'emotions': array([0, 0, 0, 0, 0, 0

 'souveraineté': {'emotions': array([0, 0, 0, 0, 0, 0]),
                  'polarity': array([1, 0])},
 'soyeux': {'emotions': array([1, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'spa': {'emotions': array([0, 0, 0, 0, 1, 0]), 'polarity': array([1, 0])},
 'spacieux': {'emotions': array([1, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'spasme': {'emotions': array([0, 1, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'spatul': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'spectacle': {'emotions': array([0, 0, 0, 0, 0, 0]),
               'polarity': array([1, 0])},
 'spectacle fabuleux': {'emotions': array([1, 0, 0, 0, 0, 0]),
                        'polarity': array([1, 0])},
 'spectaculaire': {'emotions': array([0, 0, 0, 0, 1, 0]),
                   'polarity': array([1, 0])},
 'spectateur': {'emotions': array([0, 0, 0, 0, 0, 0]),
                'polarity': array([1, 0])},
 'spectral': {'emotions': array([0, 1, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'spec

 'vraisemblance': {'emotions': array([0, 0, 0, 0, 0, 0]),
                   'polarity': array([1, 0])},
 'vriller': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([0, 1])},
 'vrombir': {'emotions': array([0, 1, 0, 0, 1, 0]), 'polarity': array([0, 1])},
 'vrombissement': {'emotions': array([0, 1, 0, 0, 1, 0]),
                   'polarity': array([0, 1])},
 'vue': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'vulgaire': {'emotions': array([0, 0, 0, 0, 0, 1]), 'polarity': array([0, 1])},
 'vulgarité': {'emotions': array([0, 0, 1, 1, 0, 1]),
               'polarity': array([0, 1])},
 'vulnérabilité': {'emotions': array([0, 1, 1, 0, 0, 0]),
                   'polarity': array([0, 1])},
 'vulnérable': {'emotions': array([0, 1, 1, 0, 0, 0]),
                'polarity': array([0, 1])},
 'véger': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'végétal': {'emotions': array([0, 0, 0, 0, 0, 0]), 'polarity': array([1, 0])},
 'végétarien':

In [19]:
import csv
import numpy as np

lemma_path = "./lexique/emotion_dictionary_lemmatized.csv"
final_path = "./lexique/emotion_dictionary_final.csv"

def process_duplicates(duplicates):
    words_to_remove = set()
    processed_duplicates = {}
    for word, info in duplicates.items():
        if len(info) > 1:  # Doublon, nous devons gérer la fusion
            polarities = np.array([info[i]['polarity'] for i in range(len(info))])
            if np.all(polarities == polarities[0]):
                # Toutes les polarités sont identiques, prenons en compte tous les "1" pour les émotions
                max_emotions = np.maximum.reduce([info[i]['emotions'] for i in range(len(info))])
                info = [{'polarity': polarities[0], 'emotions': max_emotions}]
            else:
                # Les polarités sont différentes, nous devons supprimer le mot
                words_to_remove.add(word)
        else:
            max_emotions = np.maximum.reduce([info[i]['emotions'] for i in range(len(info))])
            info = [{'polarity': info[0]['polarity'], 'emotions': max_emotions}]
        processed_duplicates[word] = info

    return words_to_remove, processed_duplicates


def write_final_dictionary(emotion_dict_path, output_file):
    duplicates = {}
    with open(emotion_dict_path, newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=';')
        for row in csv_reader:
            word = row['word']
            polarity = np.array([int(row['positive']), int(row['negative'])])
            emotions = np.array([int(row['joy']), int(row['fear']), int(row['sadness']),
                                 int(row['anger']), int(row['surprise']), int(row['disgust'])])

            if word in duplicates:
                duplicates[word].append({'polarity': polarity, 'emotions': emotions})
            else:
                duplicates[word] = [{'polarity': polarity, 'emotions': emotions}]

    # Gérer les doublons et récupérer les informations traitées
    words_to_remove, processed_duplicates = process_duplicates(duplicates)

    # Créer le dictionnaire final sans les doublons
    emotion_dict_lemmatized = {}
    for word, info in processed_duplicates.items():
        if word not in words_to_remove:
            # Prendre la première valeur de polarité et d'émotions
            polarity = info[0]['polarity']
            emotions = info[0]['emotions']
            emotion_dict_lemmatized[word] = {'polarity': polarity, 'emotions': emotions}

    # Écrire le dictionnaire d'émotions lemmatisé dans un fichier de sortie
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['word', 'positive', 'negative', 'joy', 'fear', 'sadness', 'anger', 'surprise', 'disgust']
        csv_writer = csv.DictWriter(outfile, delimiter=';', fieldnames=fieldnames)
        csv_writer.writeheader()

        for word, emotions in emotion_dict_lemmatized.items():
            row = {
                'word': word,
                'positive': emotions['polarity'][0],
                'negative': emotions['polarity'][1],
                'joy': emotions['emotions'][0],
                'fear': emotions['emotions'][1],
                'sadness': emotions['emotions'][2],
                'anger': emotions['emotions'][3],
                'surprise': emotions['emotions'][4],
                'disgust': emotions['emotions'][5]
            }
            csv_writer.writerow(row)

# Appel de la fonction pour créer le dictionnaire final
write_final_dictionary(lemma_path, final_path)


In [20]:
polarity_path = "./lexique/emotion_dictionary.csv"
clean_path = "./lexique/emotion_dictionary_clean.csv"
manual_path = "./lexique/emotion_dictionary_clean_manual.csv"
lemma_path = "./lexique/emotion_dictionary_lemmatized.csv"
final_path = "./lexique/emotion_dictionary_final.csv"

# Comptage des lignes du CSV
csv_polarity = load_csv_content(polarity_path)
csv_clean = load_csv_content(clean_path)
csv_manual = load_csv_content(manual_path)
csv_lemma = load_csv_content(lemma_path)
csv_final = load_csv_content(final_path)

num_csv_polarity = len(csv_polarity)
num_csv_clean = len(csv_clean)
num_csv_manual = len(csv_manual)
num_csv_lemma = len(csv_lemma)
num_csv_final = len(csv_final)

# Comptage des clés du dictionnaire
dict_polarity = load_dictionary(polarity_path) 
dict_clean = load_dictionary(clean_path)
dict_manual = load_dictionary(manual_path)
dict_lemma = load_dictionary(lemma_path)
dict_final = load_dictionary(final_path)

num_dict_polarity = len(dict_polarity)
num_dict_clean = len(dict_clean)
num_dict_manual = len(dict_manual)
num_dict_lemma = len(dict_lemma)
num_dict_final = len(dict_final)

print("Nombre de lignes (sans les en-têtes) :")
print("---------------------")
print(f"CSV - Gestion des polarités : {num_csv_polarity}")
print(f"CSV - Nettoyage automatique : {num_csv_clean}")
print(f"CSV - Nettoyage manuel : {num_csv_manual}")
print(f"CSV - Lemmatisation spaCy : {num_csv_lemma}")
print(f"CSV - Gestion des doublons : {num_csv_final}")
print("---------------------")
print(f"Dictionnaire - Gestion des polarités : {num_dict_polarity}")
print(f"Dictionnaire - Nettoyage automatique : {num_dict_clean}")
print(f"Dictionnaire - Nettoyage manuel : {num_dict_manual}")
print(f"Dictionnaire - Lemmatisation spaCy : {num_dict_lemma}")
print(f"Dictionnaire - Gestion des doublons : {num_dict_final}")

Nombre de lignes (sans les en-têtes) :
---------------------
CSV - Gestion des polarités : 14127
CSV - Nettoyage automatique : 14127
CSV - Nettoyage manuel : 14127
CSV - Lemmatisation spaCy : 14125
CSV - Gestion des doublons : 13624
---------------------
Dictionnaire - Gestion des polarités : 14126
Dictionnaire - Nettoyage automatique : 14110
Dictionnaire - Nettoyage manuel : 14072
Dictionnaire - Lemmatisation spaCy : 13658
Dictionnaire - Gestion des doublons : 13624


In [21]:
# Trouver les doublons dans le dictionnaire
duplicates = find_duplicates(csv_final)

# Imprimer les doublons et leurs vecteurs correspondants
for word, values in duplicates.items():
    if len(values['polarity']) > 1:
        print("---------------------")
        print("'" + word + "'")
        print("---------------------")
        for i in range(len(values['polarity'])):
            print("Polarité :", values['polarity'][i])
            print("Vecteur d'émotions :", values['emotions'][i])



#### Matrice de la distribution des termes à l'intérieur des catégories

In [22]:
# Charger les données à partir du fichier CSV dans un DataFrame
df = pd.read_csv("./lexique/emotion_dictionary_final.csv", delimiter=";", index_col=0)

df.head(10)

,positive,negative,joy,fear,sadness,anger,surprise,disgust
word,,,,,,,,
@card@ kilo,1,0,0,0,0,0,0,0
à base de plante,1,0,0,0,0,0,0,0
à bord,1,0,0,0,0,0,0,0
à capuche,1,0,0,1,0,0,0,0
à carreau,1,0,0,0,0,0,0,0
à ce endroit -là,1,0,0,0,0,0,0,0
à clou,0,1,0,0,0,0,0,0
à condition que,1,0,0,0,0,0,0,0
à contrecoeur,0,1,0,1,1,1,0,1


In [23]:
import pandas as pd

# Charger les données à partir du fichier CSV dans un DataFrame
df = pd.read_csv("./lexique/emotion_dictionary_final.csv", delimiter=";")

# Sélectionner les colonnes d'intérêt pour les émotions en utilisant l'index numérique
emotions_df = df.iloc[:, 1:]  # Ici, nous supposons que les colonnes d'intérêt commencent à l'index 1 (positivity) jusqu'à la fin

# Créer une matrice 8x8 remplie de zéros
matrix = pd.DataFrame(0, index=emotions_df.columns, columns=emotions_df.columns)

# Parcourir les termes et mettre à jour la matrice avec les occurrences des paires émotion-polarité
for term, row in emotions_df.iterrows():
    for emotion1 in emotions_df.columns:
        if row[emotion1] != 0:
            for emotion2 in emotions_df.columns:
                if row[emotion2] != 0:
                    matrix.loc[emotion1, emotion2] += 1

# Affichage de la matrice
display(matrix)


,positive,negative,joy,fear,sadness,anger,surprise,disgust
positive,8156,0,498,214,124,117,425,91
negative,0,5468,7,2862,2286,1919,723,1858
joy,498,7,505,3,0,0,6,0
fear,214,2862,3,3076,1489,1292,634,889
sadness,124,2286,0,1489,2410,907,286,811
anger,117,1919,0,1292,907,2036,351,860
surprise,425,723,6,634,286,351,1148,135
disgust,91,1858,0,889,811,860,135,1949
